In [1]:
#Knapsack Model

# Step 1 Create the weights and values
W = [4,2,5,4,5,2,3,5]
V = [10,5,18,12,15,3,2,8]
C = 15
N = len(W)

# Step 2 Importing gurobipy package
from gurobipy import *

# Step 3 Create an optimization model
knapsack_model = Model('knapsack')

# Step 4 Add decision variables
X = knapsack_model.addVars(N, vtype=GRB.BINARY, name="x")

# Step 5 Define the objective function
obj_fn = sum(V[i]*X[i] for i in range(N))
knapsack_model.setObjective(obj_fn, GRB.MAXIMIZE)

# Step 6 Add the constraints
knapsack_model.addConstr(sum(W[i]*X[i] for i in range(N)) <= C)

# Step 7 Solve the model and output the solution
knapsack_model.setParam('OutputFlag', False)
knapsack_model.optimize()
print('Optimization is done. Objective Function Value: %.2f' % knapsack_model.objVal)

for v in knapsack_model.getVars():
    print('%s: %g' % (v.varName, v.x))

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-10
Optimization is done. Objective Function Value: 45.00
x[0]: 0
x[1]: 0
x[2]: 1
x[3]: 1
x[4]: 1
x[5]: 0
x[6]: 0
x[7]: 0


In [2]:
# Simple polynomial optimization

import gurobipy as gp
from gurobipy import GRB

# Create a new model
model = gp.Model("simple_optimization")

# Create variables
x = model.addVar(name="x")
y = model.addVar(name="y")
z = model.addVar(name="z")

# Set the objective function
model.setObjective(3 * x * x + y * y - 2 * z * z, GRB.MAXIMIZE)

# Add constraints
model.addConstr(3 * x + 2 * y - 8 * z == -50, "c0")

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    for v in model.getVars():
        print(f'{v.varName}: {v.x}')
    print(f'Objective: {model.objVal}')
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i9-12900, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x9c2a78d1
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e+00, 8e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 5e+01]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 4 rows, 8 columns, 11 nonzeros
Presolved model has 1 quadratic constraint(s)
Presolved model has 2 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.

Variable types: 8 continuous, 0 integer (0 binary)

Root

In [3]:
# Transportation model

import gurobipy as gp
from gurobipy import GRB

# Define the data
sources = ['S1', 'S2']
demands = ['D1', 'D2', 'D3']
supply = {'S1': 20, 'S2': 30}
demand = {'D1': 10, 'D2': 25, 'D3': 15}
cost = {
    ('S1', 'D1'): 8, ('S1', 'D2'): 6, ('S1', 'D3'): 10,
    ('S2', 'D1'): 9, ('S2', 'D2'): 12, ('S2', 'D3'): 7
}

# Create a new model
model = gp.Model("transportation")

# Create variables
x = model.addVars(sources, demands, name="x")

# Set objective
model.setObjective(gp.quicksum(cost[s, d] * x[s, d] for s in sources for d in demands), GRB.MINIMIZE)

# Add supply constraints
model.addConstrs((gp.quicksum(x[s, d] for d in demands) <= supply[s] for s in sources), name="Supply")

# Add demand constraints
model.addConstrs((gp.quicksum(x[s, d] for s in sources) == demand[d] for d in demands), name="Demand")

# Optimize model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    for s in sources:
        for d in demands:
            if x[s, d].x > 0:
                print(f"Transport {x[s, d].x} units from {s} to {d} at cost {cost[s, d]} per unit")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i9-12900, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xab1bdb4e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.750000000e+02
Transport 20.0 units from S1 to D2 at cost 6 per unit
Transport 10.0 units from S2 to D1 at cost 9 per unit
Transport 5.0 units from S2 to D2 at cost 12 per unit
Transport 15.0 units from S2 

In [4]:
# MCLP Model

import gurobipy as gp
from gurobipy import GRB

# Define the data
I = ['D1', 'D2', 'D3', 'D4']
J = ['F1', 'F2', 'F3']
d = {
    ('D1', 'F1'): 2, ('D1', 'F2'): 4, ('D1', 'F3'): 5,
    ('D2', 'F1'): 3, ('D2', 'F2'): 2, ('D2', 'F3'): 6,
    ('D3', 'F1'): 4, ('D3', 'F2'): 3, ('D3', 'F3'): 2,
    ('D4', 'F1'): 5, ('D4', 'F2'): 6, ('D4', 'F3'): 3
}
coverage_distance = 3
S = {i: [j for j in J if d[i, j] <= coverage_distance] for i in I}
p = 2
w = {'D1': 10, 'D2': 20, 'D3': 30, 'D4': 40}

# Create a new model
model = gp.Model("MCLP")

# Create variables
x = model.addVars(J, vtype=GRB.BINARY, name="x")
y = model.addVars(I, vtype=GRB.BINARY, name="y")

# Set objective
model.setObjective(gp.quicksum(w[i] * y[i] for i in I), GRB.MAXIMIZE)

# Add constraints
model.addConstrs((y[i] <= gp.quicksum(x[j] for j in S[i]) for i in I), name="Coverage")
model.addConstr(gp.quicksum(x[j] for j in J) <= p, name="FacilityLimit")

# Optimize model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for j in J:
        if x[j].x > 0.5:
            print(f"Facility located at {j}")
    for i in I:
        if y[i].x > 0.5:
            print(f"Demand point {i} is covered")
else:
    print("No optimal solution found")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i9-12900, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 5 rows, 7 columns and 13 nonzeros
Model fingerprint: 0x95e97aca
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 5 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 2: 100 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+02, best bound 1.000000000000e+02, gap 0.0000%
Optimal solution found:
Facility located a

In [5]:
# P-Median

import gurobipy as gp
from gurobipy import GRB

# Define the data
I = ['D1', 'D2', 'D3', 'D4']
J = ['F1', 'F2', 'F3']
d = {
    ('D1', 'F1'): 2, ('D1', 'F2'): 4, ('D1', 'F3'): 5,
    ('D2', 'F1'): 3, ('D2', 'F2'): 2, ('D2', 'F3'): 6,
    ('D3', 'F1'): 4, ('D3', 'F2'): 3, ('D3', 'F3'): 2,
    ('D4', 'F1'): 5, ('D4', 'F2'): 6, ('D4', 'F3'): 3
}
p = 2

# Create a new model
model = gp.Model("p-median")

# Create variables
x = model.addVars(J, vtype=GRB.BINARY, name="x")
y = model.addVars(I, J, vtype=GRB.BINARY, name="y")

# Set objective
model.setObjective(gp.quicksum(d[i, j] * y[i, j] for i in I for j in J), GRB.MINIMIZE)

# Add constraints
model.addConstrs((gp.quicksum(y[i, j] for j in J) == 1 for i in I), name="Assign")
model.addConstrs((y[i, j] <= x[j] for i in I for j in J), name="Open")
model.addConstr(gp.quicksum(x[j] for j in J) == p, name="FacilityCount")

# Optimize model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for j in J:
        if x[j].x > 0.5:
            print(f"Facility located at {j}")
    for i in I:
        for j in J:
            if y[i, j].x > 0.5:
                print(f"Demand point {i} is assigned to facility {j}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i9-12900, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 17 rows, 15 columns and 39 nonzeros
Model fingerprint: 0x0ec5554b
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 19.0000000
Presolve time: 0.00s
Presolved: 17 rows, 15 columns, 39 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)
Found heuristic solution: objective 11.0000000

Root relaxation: objective 1.000000e+01, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    

In [6]:
# P-Center

import gurobipy as gp
from gurobipy import GRB

# Define the data
I = ['D1', 'D2', 'D3', 'D4']
J = ['F1', 'F2', 'F3']
d = {
    ('D1', 'F1'): 2, ('D1', 'F2'): 4, ('D1', 'F3'): 5,
    ('D2', 'F1'): 3, ('D2', 'F2'): 2, ('D2', 'F3'): 6,
    ('D3', 'F1'): 4, ('D3', 'F2'): 3, ('D3', 'F3'): 2,
    ('D4', 'F1'): 5, ('D4', 'F2'): 6, ('D4', 'F3'): 3
}
p = 2

# Create a new model
model = gp.Model("p-center")

# Create variables
x = model.addVars(J, vtype=GRB.BINARY, name="x")
y = model.addVars(I, J, vtype=GRB.BINARY, name="y")
z = model.addVar(vtype=GRB.CONTINUOUS, name="z")

# Set objective
model.setObjective(z, GRB.MINIMIZE)

# Add constraints
model.addConstrs((gp.quicksum(y[i, j] for j in J) == 1 for i in I), name="Assign")
model.addConstrs((y[i, j] <= x[j] for i in I for j in J), name="Open")
model.addConstr(gp.quicksum(x[j] for j in J) == p, name="FacilityCount")
model.addConstrs((d[i, j] * y[i, j] <= z for i in I for j in J), name="MaxDistance")

# Optimize model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for j in J:
        if x[j].x > 0.5:
            print(f"Facility located at {j}")
    for i in I:
        for j in J:
            if y[i, j].x > 0.5:
                print(f"Demand point {i} is assigned to facility {j}")
    print(f"Maximum distance: {z.x}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i9-12900, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 29 rows, 16 columns and 63 nonzeros
Model fingerprint: 0x10c2414d
Variable types: 1 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 21 rows, 16 columns, 49 nonzeros
Variable types: 0 continuous, 16 integer (15 binary)

Root relaxation: objective 3.000000e+00, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0   

In [7]:
# Facilities Model

from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB

# Parameters
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [3,2,3,1,3,3,4,3,2]
cost_per_mile = 1

# This function determines the Euclidean distance between a facility and customer sites.

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

# Compute key parameters of MIP model formulation

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

# Compute shipping costs

shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

# MIP  model formulation

m = gp.Model('facility_location')

select = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
assign = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='Assign')

m.addConstrs((assign[(c,f)] <= select[f] for c,f in cartesian_prod), name='Setup2ship')
m.addConstrs((gp.quicksum(assign[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

m.setObjective(select.prod(setup_cost)+assign.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

# display optimal values of decision variables

for facility in select.keys():
    if (abs(select[facility].x) > 1e-6):
        print(f"\n Build a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].x) > 1e-6):
        print(f"\n Supermarket {customer + 1} receives {round(100*assign[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i9-12900, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 20 rows, 27 columns and 54 nonzeros
Model fingerprint: 0x0939f503
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)
Found heuristic solution: objective 6.0385165

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.7237129    4.7237